In [415]:
from pybaseball import schedule_and_record, pitching_stats, batting_stats

In [416]:
import os
import pandas as pd

# Data docs at https://www.fangraphs.com/players/shohei-ohtani/19755/stats?position=DH

if os.path.exists("batting.csv"):
    batting = pd.read_csv("batting.csv", index_col=0)
else:
    batting = batting_stats(1990, 2022)
    batting.to_csv("batting.csv")

In [417]:
batting

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,Barrel%,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA
0,1109,2002,Barry Bonds,SFG,37,143,403,612,149,70,...,NaN,NaN,NaN,NaN,0,0.127,0.191,NaN,NaN,NaN
1,1109,2004,Barry Bonds,SFG,39,147,373,617,135,60,...,NaN,NaN,NaN,NaN,0,0.124,0.164,NaN,NaN,NaN
16,13611,2018,Mookie Betts,BOS,25,136,520,614,180,96,...,0.131,110.6,217.0,0.500,434,0.220,0.270,NaN,NaN,NaN
2,1109,2003,Barry Bonds,SFG,38,130,390,550,133,65,...,NaN,NaN,NaN,NaN,0,0.135,0.223,NaN,NaN,NaN
64,10155,2013,Mike Trout,LAA,21,157,589,716,190,115,...,NaN,NaN,0.0,NaN,0,0.200,0.266,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2734,1213,2009,Aubrey Huff,- - -,32,150,536,597,129,83,...,NaN,NaN,0.0,NaN,0,0.169,0.246,NaN,NaN,NaN
2679,857,2005,Bernie Williams,NYY,36,141,485,546,121,89,...,NaN,NaN,NaN,NaN,0,0.183,0.255,NaN,NaN,NaN
3038,8585,2009,Yuniesky Betancourt,- - -,27,134,470,508,115,83,...,NaN,NaN,0.0,NaN,0,0.181,0.232,NaN,NaN,NaN
3091,9272,2018,Chris Davis,BAL,32,128,470,522,79,51,...,0.096,111.8,113.0,0.401,282,0.174,0.316,NaN,NaN,NaN


In [418]:
def next_season_war(player):
    player = player.sort_values("Season")
    player["Next_WAR"] = player["WAR"].shift(-1)
    return player

batting = batting.groupby("IDfg", group_keys=False).apply(next_season_war)

In [419]:
null_count = batting.isnull().sum()

In [420]:
complete_cols = list(batting.columns[null_count == 0])
batting = batting[complete_cols + ["Next_WAR"]].copy()

In [421]:
batting

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,Pull%+,Cent%+,Oppo%+,Soft%+,Med%+,Hard%+,Events,CStr%,CSW%,Next_WAR
832,2,2002,Garret Anderson,ANA,30,158,638,678,195,107,...,118,91,80,65,97,129,0,0.137,0.232,5.1
640,2,2003,Garret Anderson,ANA,31,159,638,673,201,119,...,112,101,80,90,99,109,0,0.164,0.252,-0.2
2395,2,2005,Garret Anderson,LAA,33,142,575,603,163,111,...,126,78,83,100,97,106,0,0.150,0.258,0.1
2307,2,2006,Garret Anderson,LAA,34,141,543,588,152,105,...,113,96,86,88,94,119,0,0.155,0.253,1.4
2033,2,2008,Garret Anderson,LAA,36,145,557,593,163,118,...,121,86,84,94,103,97,0,0.175,0.258,-1.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1375,23697,2022,Julio Rodriguez,SEA,21,119,467,509,126,79,...,102,101,95,88,96,114,331,0.146,0.295,NaN
1897,24610,2022,Steven Kwan,CLE,24,121,453,513,132,103,...,74,106,133,110,121,56,408,0.254,0.284,NaN
2503,25764,2022,Bobby Witt Jr.,KCR,22,126,492,528,121,72,...,87,115,100,90,101,104,382,0.154,0.271,NaN
1511,26197,2022,Andrew Vaughn,CHW,24,111,430,469,122,81,...,88,109,107,91,100,105,358,0.206,0.291,NaN


In [422]:
batting.dtypes[batting.dtypes == "object"]

Name       object
Team       object
Dol        object
Age Rng    object
dtype: object

In [423]:
del batting["Dol"]
del batting["Age Rng"]

In [424]:
batting["Team_Code"] = batting["Team"].astype("category").cat.codes

In [425]:
batting

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,Cent%+,Oppo%+,Soft%+,Med%+,Hard%+,Events,CStr%,CSW%,Next_WAR,Team_Code
832,2,2002,Garret Anderson,ANA,30,158,638,678,195,107,...,91,80,65,97,129,0,0.137,0.232,5.1,1
640,2,2003,Garret Anderson,ANA,31,159,638,673,201,119,...,101,80,90,99,109,0,0.164,0.252,-0.2,1
2395,2,2005,Garret Anderson,LAA,33,142,575,603,163,111,...,78,83,100,97,106,0,0.150,0.258,0.1,15
2307,2,2006,Garret Anderson,LAA,34,141,543,588,152,105,...,96,86,88,94,119,0,0.155,0.253,1.4,15
2033,2,2008,Garret Anderson,LAA,36,145,557,593,163,118,...,86,84,94,103,97,0,0.175,0.258,-1.1,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1375,23697,2022,Julio Rodriguez,SEA,21,119,467,509,126,79,...,101,95,88,96,114,331,0.146,0.295,NaN,27
1897,24610,2022,Steven Kwan,CLE,24,121,453,513,132,103,...,106,133,110,121,56,408,0.254,0.284,NaN,9
2503,25764,2022,Bobby Witt Jr.,KCR,22,126,492,528,121,72,...,115,100,90,101,104,382,0.154,0.271,NaN,14
1511,26197,2022,Andrew Vaughn,CHW,24,111,430,469,122,81,...,109,107,91,100,105,358,0.206,0.291,NaN,7


In [426]:
batting_full = batting.copy()
batting = batting.dropna()

In [427]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import TimeSeriesSplit

rf = RandomForestRegressor(n_estimators=10, random_state=1)

split = TimeSeriesSplit(n_splits=2)

sfs = SequentialFeatureSelector(rf, 
                                n_features_to_select=20, 
                                direction="forward",
                                cv=split,
                                n_jobs=8
                               )

In [428]:
removed_columns = ["Next_WAR", "Name", "Team", "IDfg", "Season"]
selected_columns = batting.columns[~batting.columns.isin(removed_columns)]

In [429]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

batting[selected_columns] = scaler.fit_transform(batting[selected_columns])

/var/folders/xz/9z84c__j28g8tg28bmcthjj00000gn/T/ipykernel_70648/3281851671.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batting[selected_columns] = scaler.fit_transform(batting[selected_columns])


In [430]:
sfs.fit(batting[selected_columns], batting["Next_WAR"])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=2, test_size=None),
                          estimator=RandomForestRegressor(n_estimators=10,
                                                          random_state=1),
                          n_features_to_select=20, n_jobs=8)

In [431]:
predictors = list(selected_columns[sfs.get_support()])

In [432]:
predictors

['PA',
 'SH',
 'SLG',
 'GB%',
 'FB%',
 'IFFB%',
 'BUH%',
 'WAR',
 'wRC+',
 'pLI',
 'Clutch',
 'wSL',
 'Z-Contact%',
 'SwStr%',
 'Def',
 'OBP+',
 'GB%+',
 'FB%+',
 'Cent%+',
 'Events']

In [433]:
def backtest(data, model, predictors, start=5, step=1):
    all_predictions = []
    
    years = sorted(data["Season"].unique())
    
    for i in range(start, len(years), step):
        current_year = years[i]
        train = data[data["Season"] < current_year]
        test = data[data["Season"] == current_year]
        
        model.fit(train[predictors], train["Next_WAR"])
        
        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index=test.index)
        combined = pd.concat([test["Next_WAR"], preds], axis=1)
        combined.columns = ["actual", "prediction"]
        
        all_predictions.append(combined)
    return pd.concat(all_predictions)

In [434]:
from xgboost import XGBRegressor

xgb = XGBRegressor(random_state=1, learning_rate=.01, n_estimators=200, n_jobs=8)

predictions = backtest(batting, xgb, predictors)

In [435]:
predictions.shape

(1678, 2)

In [436]:
from sklearn.metrics import mean_squared_error

mean_squared_error(predictions["actual"], predictions["prediction"])

3.7573148504305856

In [437]:
batting["WAR"].describe()

count    2310.000000
mean        0.387818
std         0.130746
min         0.000000
25%         0.301282
50%         0.378205
75%         0.467949
max         1.000000
Name: WAR, dtype: float64

In [438]:
ratios = [p for p in predictors if p not in ["Events", "Clutch"]]

In [439]:
batting = batting_full.copy().dropna()
batting = batting.groupby("IDfg", group_keys=False).filter(lambda x: x.shape[0] > 1)

In [440]:
def player_history(df):
    df = df.sort_values("Season")
    
    for col in ratios:
        prev_col = df[col].shift(1)
        df[f"{col}_prev_ratio"] = df[col] / (prev_col + .01)
        
    df["player_season"] = range(0, df.shape[0])
    df["war_corr"] = list(df[["player_season", "WAR"]].expanding().corr().droplevel(0).loc["player_season","WAR"])
    df["injured"] = (df["G"] < df["G"].shift(1)).astype(int)
    
    return df.iloc[1:,:]

batting = batting.groupby("IDfg", group_keys=False).apply(player_history)

In [441]:
batting

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,Z-Contact%_prev_ratio,SwStr%_prev_ratio,Def_prev_ratio,OBP+_prev_ratio,GB%+_prev_ratio,FB%+_prev_ratio,Cent%+_prev_ratio,player_season,war_corr,injured
640,2,2003,Garret Anderson,ANA,31,159,638,673,201,119,...,1.018931,0.838095,-0.444994,1.029897,1.181684,0.962527,1.109768,1,1.000000,0
2395,2,2005,Garret Anderson,LAA,33,142,575,603,163,111,...,0.952432,1.112245,-4.626039,0.902825,0.903759,1.029026,0.772201,2,-0.709991,1
2307,2,2006,Garret Anderson,LAA,34,141,543,588,152,105,...,1.020202,0.823529,0.659077,1.021396,1.010531,0.943307,1.230611,3,-0.788846,1
2033,2,2008,Garret Anderson,LAA,36,145,557,593,163,118,...,1.002176,0.768519,0.227480,1.020945,1.031470,0.989901,0.895740,4,-0.657658,0
2857,10,2003,David Eckstein,ANA,28,120,452,517,114,88,...,0.981366,0.836735,0.735156,0.881738,1.174643,0.955449,1.030507,1,-1.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2782,19470,2021,Tommy Edman,STL,26,159,641,691,168,113,...,1.015351,0.744186,1.672026,0.959086,0.907487,1.202540,0.793035,1,1.000000,0
1673,19611,2020,Vladimir Guerrero Jr.,TOR,21,60,221,243,58,34,...,0.952381,0.982759,0.375213,0.980859,1.093924,0.857049,0.935694,1,1.000000,1
78,19611,2021,Vladimir Guerrero Jr.,TOR,22,161,604,698,188,110,...,0.988235,1.000000,2.306525,1.232890,0.820248,1.256249,0.999902,2,0.892171,0
3,20123,2020,Juan Soto,WSN,21,47,154,196,54,27,...,1.014840,0.650000,-12.580645,1.247831,1.236986,0.771355,1.262499,1,-1.000000,1


In [442]:
new_predictors = predictors + ["player_season", "war_corr", "injured"] + [f"{col}_prev_ratio" for col in ratios]

In [443]:
predictions = backtest(batting, xgb, new_predictors)

In [444]:
mean_squared_error(predictions["actual"], predictions["prediction"]) 

3.7432965142113925